In [ ]:
import os
import numpy
# os.path.exists(os.path.join(os.getcwd(), 'new_folder', 'file.txt'))
p = os.path.join(os.getcwd(), 'weights')

In [ ]:
dir_path = "./hello/world"

In [ ]:
if not os.path.exists(path=dir_path):
    os.makedirs(name=dir_path)

In [ ]:
os.path.dirname(p)

In [ ]:
def load_numpy(file_path):
    if os.path.exists(path=file_path):
        return np.load(file_path)
    else:
        print('file not exist and return empty np array.')
        return np.array([])

In [1]:
from utils.utils import create_robot_dirs, remove_empty_robot_dirs

In [2]:
create_robot_dirs()

Create ./data/tmp/
Create ./weights/tmp/
Create ./data/tmp/trajectory/
Create ./data/tmp/show_pose/


In [4]:
remove_empty_robot_dirs()

Delete ./data/tmp/show_pose
Delete ./data/tmp/trajectory
Delete ./weights/tmp/


In [1]:
import numpy as np

In [2]:
k = np.zeros(shape=(4, 4))

In [3]:
k.shape == (4,4)

True